CNN

In [3]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [4]:
device = torch.device("xpu" if torch.xpu.is_available() else 'cpu')
print(f"Device : {device}")

Device : xpu


In [5]:
train = pd.read_csv("D:\\Python\\cnn\\fashion-mnist_train.csv")
test = pd.read_csv("D:\\Python\\cnn\\fashion-mnist_test.csv")
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train.shape

(60000, 785)

In [7]:
x_train , y_train = train.iloc[:,1:],train.iloc[:,0]
x_test , y_test = test.iloc[:,1:],test.iloc[:,0]

In [8]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features = torch.tensor(features.values,dtype=torch.float32).reshape(-1,1,28,28)
        # reshape the dataset -> -1 for specifying the batch size afterwards 
        #                     -> 1 for specifing that its a grayscale image , 3 for rgb -> channels
        #                     -> 28x28 = 784 i.e. the no of input features
        self.labels = torch.tensor(labels.values,dtype=torch.long)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index] , self.labels[index]

In [16]:
train_dataset = CustomDataset(x_train,y_train)
test_dataset = CustomDataset(x_test,y_test)

In [17]:
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False)

In [19]:
class myNN(nn.Module): # CNN Architecture has 2 main components : feture extractor and ann(classification)
    def __init__(self,input_features):
        super().__init__()
        # feature extraction -> has convolutional layers and pooling layers
        self.features = nn.Sequential(
            nn.Conv2d(input_features,32,kernel_size=3,padding='same'), 
            # input_features = 1 for grayscale, 3 for rgb
            # 32 is the number of output channels (filters)
            # kernel_size = 3 means 3x3 filter
            # padding='same' means output size is same as input size
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(32,64,kernel_size=3,padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
        # classification
        self.classifier = nn.Sequential(nn.Flatten(),nn.Linear(64*7*7,128),nn.ReLU(), nn.Dropout(0.4),
                                        nn.Linear(128,64), nn.ReLU(), nn.Dropout(0.4),
                                        nn.Linear(64,10)) # 64*7*7 is the output size after the feature extraction

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [20]:
lr = 0.07
epochs = 100
model = myNN(1).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=lr,weight_decay=1e-4)

In [21]:
# training loop

for i in range(epochs):
    total_loss = 0
    for batch_features,batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        output = model(batch_features)
        loss = loss_fn(output,batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {i+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

Epoch 1/100, Loss: 0.5225362435841103
Epoch 2/100, Loss: 0.35043226971999925
Epoch 3/100, Loss: 0.305622343212239
Epoch 4/100, Loss: 0.2744611113993471
Epoch 5/100, Loss: 0.25301721661107374
Epoch 6/100, Loss: 0.2313512478079369
Epoch 7/100, Loss: 0.22051131473913757
Epoch 8/100, Loss: 0.20712859803902062
Epoch 9/100, Loss: 0.1966456086762043
Epoch 10/100, Loss: 0.18159871034498917
Epoch 11/100, Loss: 0.17416940440437687
Epoch 12/100, Loss: 0.16695363888703685
Epoch 13/100, Loss: 0.1561730509993237
Epoch 14/100, Loss: 0.1526615099548531
Epoch 15/100, Loss: 0.14164007254548547
Epoch 16/100, Loss: 0.13728402132716483
Epoch 17/100, Loss: 0.13052894480661478
Epoch 18/100, Loss: 0.1256845431116376
Epoch 19/100, Loss: 0.1217950391722148
Epoch 20/100, Loss: 0.11233374788892517
Epoch 21/100, Loss: 0.11330621797285641
Epoch 22/100, Loss: 0.1096463120137371
Epoch 23/100, Loss: 0.10414244446740635
Epoch 24/100, Loss: 0.09888826751275294
Epoch 25/100, Loss: 0.09896007781999229
Epoch 26/100, Loss: 

In [22]:
model.eval()
total=0
correct = 0
with torch.no_grad():
    for batch_features,batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        output = model(batch_features)
        _, predicted = torch.max(output, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 93.05%


In [23]:
# evaluation on training set
model.eval()
total_train = 0
correct_train = 0
with torch.no_grad():
    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        output = model(batch_features)
        _, predicted = torch.max(output, 1)
        total_train += batch_labels.size(0)
        correct_train += (predicted == batch_labels).sum().item()
    print(f"Train Accuracy: {100 * correct_train / total_train:.2f}%")

Train Accuracy: 99.86%
